In [1]:
pip install ecpy

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pycryptodome

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install sympy

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install requests

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [30]:
import math
import time
import random
import sympy
import warnings
from random import randint, seed
import sys
from ecpy.curves import Curve,Point
from Crypto.Hash import SHA3_256, HMAC, SHA256
import requests
from Crypto.Cipher import AES
from Crypto import Random
from Crypto.Util.Padding import pad
from Crypto.Util.Padding import unpad
import random
import re
import json

from random import randint, seed
from ecpy.curves import Curve
from Crypto.Hash import SHA3_256
from Crypto import Random   # a bit better secure random number generation 
import math

In [32]:
# IK data emre
sA = 75365099691681163466259435055151388598990196366045037773536006665067396329561
Q_x = 87058090705547232336742506266448902457032608537577399368869745072849975094263
Q_y = 73863361865121729253370141189521784002483174318560123943873444021233722393171

s_ik = 38435919104545028465405229083851199426577612833328471419512121279897034838259
h_ik = 40139398399024429188254655216662123008175195428486006903289784284292094948763

# SPK data
spk_private = 49760804016305672242628858452671561631883295017037579045439043666340119331411
spk_pub_x = 13279412479756039023006939294151267045839717479408817839293710814463378950452
spk_pub_y = 93617390663867776116902322986952603938589221775532373175859757173961357675363

h_spk = 70079813329073930856804606363140448062677695844504871159148958480169809247078
s_spk = 42702578452500864554658062321887469197282606023250231586097905168842648683915

# OTK data


In [42]:
# IK data (ekin)
stuID = 26846
sA = 26037317628054985808039282432726226222027022467068096985918786230345691957027
Q_x = 34966638871069348443434840366429984004050324904300954986624560986102166574831
Q_y = 18659130989987264139111259616350811871077628350857638087893010137535414968071

s_ik = 10711401428822947968805605574831727095538167654217243597237904326250472141120
h_ik = 2310173870477934111899724596823579375648537282076021725587322962459991319407

# SPK data
spk_private = 108276252172224876587334255735165101721975917778097283393900202750918785959710
spk_pub_x = 97710353570137819193379937963704253670763857898842070763814037579480279837106
spk_pub_y = 23341606178871725188404066641975812905580291605280961572149830211055662093589

h_spk = 50104479202124469309008236680532873175969290556870197285989963025491089808550
s_spk = 56354363435458804487927205429170920223354636346201640247065025929905617989041

# OTK data


In [43]:
E = Curve.get_curve('secp256k1')
n = E.order
p = E.field
P = E.generator
a = E.a
b = E.b
print("Base point:\n", P)
print("p :", p)
print("a :", a)
print("b :", b)
print("n :", n)

k = Random.new().read(int(math.log(n,2)))
k = int.from_bytes(k, byteorder='big')%n

Q = k*P
print("\nQ:\n", Q)
print("Q on curve?", E.is_on_curve(Q))

Base point:
 (0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798 , 0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8)
p : 115792089237316195423570985008687907853269984665640564039457584007908834671663
a : 0
b : 7
n : 115792089237316195423570985008687907852837564279074904382605163141518161494337

Q:
 (0x9ceca32cc21eb150f4e19568a63909a10ec6354a644ec63de7aa540fac735ea6 , 0x167329256c81b7dd3164421f868c1e4bcf33e93edb51757f8565a8d3f216c2b8)
Q on curve? True


In [36]:
API_URL = 'http://10.92.52.175:5000/'

stuID =  25139  ## Change this to your ID number

#server's Identitiy public key
IKey_Ser = Point(93223115898197558905062012489877327981787036929201444813217704012422483432813 , 8985629203225767185464920094198364255740987346743912071843303975587695337619, E)

#Send Public Identitiy Key Coordinates and corresponding signature
def IKRegReq(h,s,x,y):
    mes = {'ID':stuID, 'H': h, 'S': s, 'IKPUB.X': x, 'IKPUB.Y': y}
    print("Sending message is: ", mes)
    response = requests.put('{}/{}'.format(API_URL, "IKRegReq"), json = mes)		
    if((response.ok) == False): print(response.json())

#Send the verification code
def IKRegVerify(code):
    mes = {'ID':stuID, 'CODE': code}
    print("Sending message is: ", mes)
    response = requests.put('{}/{}'.format(API_URL, "IKRegVerif"), json = mes)
    if((response.ok) == False): raise Exception(response.json())
    print(response.json())

#Send SPK Coordinates and corresponding signature
def SPKReg(h,s,x,y):
    mes = {'ID':stuID, 'H': h, 'S': s, 'SPKPUB.X': x, 'SPKPUB.Y': y}
    print("Sending message is: ", mes)
    response = requests.put('{}/{}'.format(API_URL, "SPKReg"), json = mes)		
    if((response.ok) == False): 
        print(response.json())
    else: 
        res = response.json()
        return res['SPKPUB.X'], res['SPKPUB.Y'], res['H'], res['S']

#Send OTK Coordinates and corresponding hmac
def OTKReg(keyID,x,y,hmac):
    mes = {'ID':stuID, 'KEYID': keyID, 'OTKI.X': x, 'OTKI.Y': y, 'HMACI': hmac}
    print("Sending message is: ", mes)
    response = requests.put('{}/{}'.format(API_URL, "OTKReg"), json = mes)		
    print(response.json())
    if((response.ok) == False): return False
    else: return True

#Send the reset code to delete your Identitiy Key
#Reset Code is sent when you first registered
def ResetIK(rcode):
    mes = {'ID':stuID, 'RCODE': rcode}
    print("Sending message is: ", mes)
    response = requests.delete('{}/{}'.format(API_URL, "ResetIK"), json = mes)		
    print(response.json())
    if((response.ok) == False): return False
    else: return True

#Sign your ID  number and send the signature to delete your SPK
def ResetSPK(h,s):
    mes = {'ID':stuID, 'H': h, 'S': s}
    print("Sending message is: ", mes)
    response = requests.delete('{}/{}'.format(API_URL, "ResetSPK"), json = mes)		
    print(response.json())
    if((response.ok) == False): return False
    else: return True

#Send the reset code to delete your Identitiy Key
def ResetOTK(h,s):
    mes = {'ID':stuID, 'H': h, 'S': s}
    print("Sending message is: ", mes)
    response = requests.delete('{}/{}'.format(API_URL, "ResetOTK"), json = mes)		
    if((response.ok) == False): print(response.json())



In [28]:
# IK Generation
sA = random.randint(1, n-2) # random secret key
Q = sA * P  # generating public key
Q_x = Q.x
Q_y = Q.y
print("sA:", sA)
print("Q:", Q)
print("Q.x:", Q_x)
print("Q.y:", Q_y)

sA: 75365099691681163466259435055151388598990196366045037773536006665067396329561
Q: (0xc07922171f0674d52f11ce7abbc154557ddce928621c74f2aa15605b8e01abf7 , 0xa34d2e6967d58f9d312430908ec537839ffc50322d80ee3f99feaed9e3250253)
Q.x: 87058090705547232336742506266448902457032608537577399368869745072849975094263
Q.y: 73863361865121729253370141189521784002483174318560123943873444021233722393171


In [29]:
# Signature Generation for IK
k = random.randint(1, n-2)
R = k * P
r = R.x % n

r_byte_array = r.to_bytes((r.bit_length() +7)//8, byteorder = 'big')
m_byte_array = stuID.to_bytes((stuID.bit_length() +7)//8, byteorder = 'big')
r_m = r_byte_array + m_byte_array #concatenation of r and m where m is stuID
print(r_byte_array)
print(m_byte_array)
print(r_m)


h = SHA3_256.new(r_m)
h = int.from_bytes(h.digest(), byteorder='big') % n


s = (k - (sA * h)) % n
print("s:", s)
print("h:", h)

b'ZU\xfc\xea\x17\xde\x9e\x80\xb08\xb0\xaf]\xd2\xe2\xfe\xc6\xffk\x06\xb2tq\xa4\xcc{\x14\xd9 y\x1c\xb8'
b'b3'
b'ZU\xfc\xea\x17\xde\x9e\x80\xb08\xb0\xaf]\xd2\xe2\xfe\xc6\xffk\x06\xb2tq\xa4\xcc{\x14\xd9 y\x1c\xb8b3'
s: 38435919104545028465405229083851199426577612833328471419512121279897034838259
h: 40139398399024429188254655216662123008175195428486006903289784284292094948763


In [30]:
IKRegReq(h, s, Q_x, Q_y)

Sending message is:  {'ID': 25139, 'H': 40139398399024429188254655216662123008175195428486006903289784284292094948763, 'S': 38435919104545028465405229083851199426577612833328471419512121279897034838259, 'IKPUB.X': 87058090705547232336742506266448902457032608537577399368869745072849975094263, 'IKPUB.Y': 73863361865121729253370141189521784002483174318560123943873444021233722393171}


In [31]:
CODE = 383538
IKRegVerify(CODE)

Sending message is:  {'ID': 25139, 'CODE': 383538}
Registered successfully


In [32]:
RCODE = 903764

In [38]:
# SPK Generation
spk_private = random.randint(1, n-2) # random secret key
spk_pub = spk_private * P#Q  # generating public key
spk_pub_x = spk_pub.x
spk_pub_y = spk_pub.y
print("spk_private:", spk_private)
print("spk_pub:", spk_pub)
print("spk_pub_x:", spk_pub_x)
print("spk_pub_y:", spk_pub_y)

# Signature Generation for SPK

# calculating to be signed message
spk_pub_x_byte_array = spk_pub_x.to_bytes((spk_pub_x.bit_length() +7)//8, byteorder = 'big')
spk_pub_y_byte_array = spk_pub_y.to_bytes((spk_pub_y.bit_length() +7)//8, byteorder = 'big')
spk_x_y = spk_pub_x_byte_array + spk_pub_y_byte_array #concatenation of spk.x and spk.y 
m = spk_x_y
#print("spk_pub_x_byte_array", spk_pub_x_byte_array)
#print("spk_pub_y_byte_array", spk_pub_y_byte_array)
#print("concatenation of spk.x and spk.y:", spk_x_y)

# signature generation
k = random.randint(1, n-2)

R = k * P #P # Q is public IK
r = R.x % n
r_byte_array = r.to_bytes((r.bit_length() +7)//8, byteorder = 'big')
r_m = r_byte_array + m # concatenation of r and m where m is concatenation of spk.x and spk.y 


h = SHA3_256.new(r_m)
h = int.from_bytes(h.digest(), byteorder='big') % n

s = (k - (sA * h)) % n

print("h:", h)
print("s:", s)

spk_private: 49760804016305672242628858452671561631883295017037579045439043666340119331411
spk_pub: (0x1d5be19079695e00267445161e44c526fd0dd6857f89f7724b1626c88bbd8934 , 0xcef990bc9e04723ad52fdf38d99436ce564539b573349ae039dcbe3218d24b63)
spk_pub_x: 13279412479756039023006939294151267045839717479408817839293710814463378950452
spk_pub_y: 93617390663867776116902322986952603938589221775532373175859757173961357675363
h: 70079813329073930856804606363140448062677695844504871159148958480169809247078
s: 42702578452500864554658062321887469197282606023250231586097905168842648683915


In [39]:
SPKReg(h, s, spk_pub_x, spk_pub_y)

Sending message is:  {'ID': 25139, 'H': 70079813329073930856804606363140448062677695844504871159148958480169809247078, 'S': 42702578452500864554658062321887469197282606023250231586097905168842648683915, 'SPKPUB.X': 13279412479756039023006939294151267045839717479408817839293710814463378950452, 'SPKPUB.Y': 93617390663867776116902322986952603938589221775532373175859757173961357675363}


(85040781858568445399879179922879835942032506645887434621361669108644661638219,
 46354559534391251764410704735456214670494836161052287022185178295305851364841,
 59007954485099251624961572416014350736776821380203722506041937308252698607740,
 101911760621645665479179352120241706761371492322672083429040706302642553897152)

In [44]:
server_SPK_pub = (85040781858568445399879179922879835942032506645887434621361669108644661638219,
 46354559534391251764410704735456214670494836161052287022185178295305851364841,
 59007954485099251624961572416014350736776821380203722506041937308252698607740,
 101911760621645665479179352120241706761371492322672083429040706302642553897152)

server_H = server_SPK_pub[2]
server_S = server_SPK_pub[3]
server_SPK_pub_x = server_SPK_pub[0]
server_SPK_pub_y = server_SPK_pub[1]


#Signature Verification:
server_spk_pub_x_byte_array = server_SPK_pub_x.to_bytes((server_SPK_pub_x.bit_length() +7)//8, byteorder = 'big')
server_spk_pub_y_byte_array = server_SPK_pub_y.to_bytes((server_SPK_pub_y.bit_length() +7)//8, byteorder = 'big')
server_spk_x_y = server_spk_pub_x_byte_array + server_spk_pub_y_byte_array # concatenation of server_spk.x and server_spk.y 

V = (server_S*P) + (server_H*IKey_Ser) 
v = V.x % n
v_byte_array = v.to_bytes((v.bit_length() +7)//8, byteorder = 'big')
v_m = v_byte_array + server_spk_x_y
h_ = SHA3_256.new(v_m)
h_ = int.from_bytes(h_.digest(), byteorder='big') % n

print("h_:", h_)
print("h :", server_H)
if(server_H == h_):
    print("verified")
else:
    print("not verified")


h_: 59007954485099251624961572416014350736776821380203722506041937308252698607740
h : 59007954485099251624961572416014350736776821380203722506041937308252698607740
verified


In [45]:
# Generating HMAC Key (KHMAC):
server_SPK_pub_points = Point(server_SPK_pub_x, server_SPK_pub_y, curve = E)
T = spk_private * server_SPK_pub_points
T_x = T.x
T_y = T.y

T_x_byte_array = T_x.to_bytes((T_x.bit_length() +7)//8, byteorder = 'big')
T_y_byte_array = T_y.to_bytes((T_y.bit_length() +7)//8, byteorder = 'big')
joker = bytes("NoNeedToRideAndHide", 'utf-8')
U = T_x_byte_array + T_y_byte_array + joker

K_HMAC = SHA3_256.new(U)

print("T_x_byte_array:", T_x_byte_array)
print("T_y_byte_array:", T_y_byte_array)
print("U:", U)
print("K_HMAC:", K_HMAC.digest())

T_x_byte_array: b'\xd6\xbby\xd0\xe6\xc3\xe1\xd1\xef\xae\xfb/\x8f\x9a\x1b\x01P\x89\xd6z\x03\xa1\xcfc\xd6ZD/\x05\xe7\x1a\xac'
T_y_byte_array: b'\xb3\xa4\x01\xab\xc0\xce\xb5\xc24\x8dE\xd0jpw\x17\xebz\xab\xf1\xb0\xc9\xf2\x9a\x90\x94\x80\xb1\xb4\xee\xd6\xda'
U: b'\xd6\xbby\xd0\xe6\xc3\xe1\xd1\xef\xae\xfb/\x8f\x9a\x1b\x01P\x89\xd6z\x03\xa1\xcfc\xd6ZD/\x05\xe7\x1a\xac\xb3\xa4\x01\xab\xc0\xce\xb5\xc24\x8dE\xd0jpw\x17\xebz\xab\xf1\xb0\xc9\xf2\x9a\x90\x94\x80\xb1\xb4\xee\xd6\xdaNoNeedToRideAndHide'
K_HMAC: b"\xb3'\xf2\xda\xa7!I\x18\x8c\xbe\xdf\xa4\x12F3.\x1f/\xc6\xea1\x05\xa6\x80\xf9\xbe\x7fX\x14\xfd3,"


In [46]:
# OTKs generation
hmac_array = [] 
OTK_public = []
OTK_private = []

for i in range(10):
    print("for ID:", i)
    otk_private = random.randint(1, n-2) # random secret key
    otk_pub = otk_private * P # generating public key

    otk_pub_x = otk_pub.x
    otk_pub_y = otk_pub.y

    print("private:", otk_private)
    print("public:", otk_pub)
    print("public_x:", otk_pub_x)
    print("public_y:", otk_pub_y)

    otk_pub_x_byte_array = otk_pub_x.to_bytes((otk_pub_x.bit_length() +7)//8, byteorder = 'big')
    otk_pub_y_byte_array = otk_pub_y.to_bytes((otk_pub_y.bit_length() +7)//8, byteorder = 'big')
    otk_public_x_y = otk_pub_x_byte_array + otk_pub_y_byte_array #concatenation of x and y 

    hmac = HMAC.new(key = K_HMAC.digest(), msg = otk_public_x_y, digestmod = SHA256)
    
    print("hmac.hexdigest():", hmac.hexdigest())
    hmac_array.append(hmac)
    OTK_public.append(otk_pub)
    OTK_private.append(otk_private)

    OTKReg(i, otk_pub_x, otk_pub_y, hmac.hexdigest())
    print('------------------------------------------')


for ID: 0
private: 94377182566090365304330189797525479447070004536023737279250018880938850525785
public: (0x602ad89e8dba9d27778e47380d0af238b4f81113e72eabb2e876e134b9e50441 , 0x4441c715053a7541fea7bf47d88b5b5cda2d19088157d93da09ccae546d8e261)
public_x: 43497736092522547221454206138770411412167587358545040689354162069648252929089
public_y: 30873492777105210423596955824875935419532639965816320531049714567313190609505
hmac.hexdigest(): cb2615197affd067eb7695420f3920d9e33b57308e154623afbd4ec8f773182b
Sending message is:  {'ID': 26846, 'KEYID': 0, 'OTKI.X': 43497736092522547221454206138770411412167587358545040689354162069648252929089, 'OTKI.Y': 30873492777105210423596955824875935419532639965816320531049714567313190609505, 'HMACI': 'cb2615197affd067eb7695420f3920d9e33b57308e154623afbd4ec8f773182b'}
OTK with ID number0 is registered successfully
------------------------------------------
for ID: 1
private: 89101635162612515839188692443378965614053378522540636366523253843333293985635
public: (

In [47]:
print("hmac keys:",hmac_array)
print("Public OTK:",OTK_public)
print("Private OTK:",OTK_private)

hmac keys: [<Crypto.Hash.HMAC.HMAC object at 0x11234d840>, <Crypto.Hash.HMAC.HMAC object at 0x11234f5b0>, <Crypto.Hash.HMAC.HMAC object at 0x11234f250>, <Crypto.Hash.HMAC.HMAC object at 0x11234d900>, <Crypto.Hash.HMAC.HMAC object at 0x11234f670>, <Crypto.Hash.HMAC.HMAC object at 0x11234f310>, <Crypto.Hash.HMAC.HMAC object at 0x11234dde0>, <Crypto.Hash.HMAC.HMAC object at 0x10f676680>, <Crypto.Hash.HMAC.HMAC object at 0x11234c160>, <Crypto.Hash.HMAC.HMAC object at 0x10f69ba60>]
Public OTK: [<ecpy.curves.Point object at 0x110a06ec0>, <ecpy.curves.Point object at 0x111784600>, <ecpy.curves.Point object at 0x10f5e5740>, <ecpy.curves.Point object at 0x111039700>, <ecpy.curves.Point object at 0x1110191c0>, <ecpy.curves.Point object at 0x111d85ac0>, <ecpy.curves.Point object at 0x11177a3c0>, <ecpy.curves.Point object at 0x111789380>, <ecpy.curves.Point object at 0x11177b040>, <ecpy.curves.Point object at 0x1115e3a40>]
Private OTK: [9437718256609036530433018979752547944707000453602373727925001

In [1]:
import math
import time
import random
import sympy
import warnings
from random import randint, seed
import sys
from ecpy.curves import Curve,Point
from Crypto.Hash import SHA3_256
import requests
from Crypto.Cipher import AES
from Crypto import Random
from Crypto.Util.Padding import pad
from Crypto.Util.Padding import unpad
import random
import re
import json
import argparse


API_URL = '10.92.52.175:6000'


def view_status():
    try:
        response = requests.get(f'http://{API_URL}')		
        if((response.ok) == False):
            raise Exception(response.json())
        else:
            print(response.json())
    except Exception as e:
        print(e)

def view_classlist():
    try:
        response = requests.get(f'http://{API_URL}/Class')		
        if((response.ok) == False):
            raise Exception(response.json())
        else:
            print(response.json())
    except Exception as e:
        print(e)



def register(id_, rand_nam):
    try:
        mes = {'ID': id_, 'random': rand_nam}
        response = requests.put(f'http://{API_URL}/Register', json = mes)
        if((response.ok) == False):
            raise Exception(response.json())
        else:
            print(response.json())
    except Exception as e:
        print(e)
        




stuID = 25139	 ##Change this to your ID number
num = 13998852341340 ##Create a random number or enter your favourite num

#Register to the most secure lottery ever:D
#register(stuID, num)

#View status
view_status()

#Comment out to view class list 
#view_classlist()


HTTPConnectionPool(host='10.92.52.175', port=6000): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1115c5e70>: Failed to establish a new connection: [Errno 61] Connection refused'))
